# Modeling with a Graph Neural Network

Now, we pose this as a vertex regression task.  That is, we have a graph $G = (V, E)$ with associated vertices $V = \{1, 2, \ldots, 456 \}$ representing locations and edges $E$ representing nearest neighbor relationships.  Each vertex $v \in V$ is also associated with an input feature $x_v \in \mathbb{R}$ representing the (scaled) altitude as well as a target value $y_v \in \mathbb{R}$ representing the precipitation.  We will use a [Graph Convolutional Network](https://arxiv.org/abs/1609.02907) to predict the $y_v$ values.

## Loading the data

We will load the data pre-processed in R into a Pytorch Geometric InMemoryDataset using the [convert_pytorch_geometric](https://github.com/rmurphy2718/gnn-spatial-exploration/blob/main/convert_pytorch_geometric.py) script.  

In [1]:
import torch
import torch_geometric
import torch_geometric.transforms as transforms
from convert_pytorch_geometric import CaRain, create_pyg

torch.manual_seed(42)  # Set a seed for consistency in blogging.

In [2]:
dataset = CaRain(".", pre_transform=create_pyg)
g = dataset[0]

# Note: nearest neighbors graph is not regular; that is, not all vertices have the same degree.
# So, we can add a one-hot encoding of the vertex degree as an additional feature as is sometimes done with featureless graphs, 
# in case it carries extra useful information.
degrees = torch_geometric.utils.degree(g.edge_index[0])
max_deg = torch.max(degrees).long().item()
add_degree_fn = transforms.OneHotDegree(max_deg)
add_degree_fn(g)


Data(edge_index=[2, 2772], x=[456, 13], y=[456])

Next, we partition the vertex set into train and test sets.  We will leave it at that, although using multiple random sets and model initializations would provide a better assessment of generalization performance.  I will update this blog if I make this enhancement.

Specifically, we create boolean indicators -- masks -- that indicate whether a vertex belongs to a given set. 

In [3]:
def make_train_test_masks(data_size, train_size):
    # Creat a boolean with exactly `train_size` are True, rest False
    unshuffled_train = int(train_size) > torch.arange(data_size)
    
    # shuffle to get train mask
    train_mask = unshuffled_train[torch.randperm(data_size)]
    
    # negate to get test mask
    test_mask = ~train_mask
    
    return train_mask, test_mask

g.train_mask, g.test_mask = make_train_test_masks(g.num_nodes, 0.2 * g.num_nodes)

## Train a GNN model

Next, we initialize an off-the-shelf Graph Convolutional Network given in the [PyTorch Geometric tutorial](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html).  I wonder whether this is enough to do better than the global regression?  In my experience, while refined Message Passing GNNs have been developed since GCN, it performs reasonably well and is a simple choice.

In [4]:
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GCNConv

class Model(nn.Module):
    def __init__(self, in_dim):
        super(Model, self).__init__()
        self.conv1 = GCNConv(in_dim, 16)
        self.conv2 = GCNConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.25, training=self.training)  # In my experience, weaker dropout is good
        x = self.conv2(x, edge_index)

        return x.squeeze()

We train the model using the Adam optimizer for 5000 epochs.  Even on my local machine's CPUs, it only takes a few moments to train.

In [5]:
model = Model(g.num_node_features)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay=5e-4)
crit = nn.MSELoss()

model.train()
for epo in range(5000):
    optimizer.zero_grad()
    pred = model(g)
        
    loss = crit(pred[g.train_mask], g.y[g.train_mask])
    loss.backward()
    optimizer.step()
    
    if epo % 500 == 0:
        print(loss.item())


515239.75
162703.046875
150588.046875
162187.421875
152907.640625
168840.875
156450.796875
153671.765625
145880.078125
145020.8125


Note that the MSE values are large due to the scale of the target.  Finally, we can evaluate the model on the vertices in the test set.

In [6]:
model.eval()
test_pred = model(g)
test_loss = crit(pred[g.test_mask], g.y[g.test_mask]).item()
print(test_loss)

184472.328125


# Compare with global linear regression

The [Spatial Data Anlysis textbook](https://rspatial.org/raster/analysis/analysis.pdf) uses (global) linear regression as the naive baseline.  Let us see how it performs.

In [7]:
from sklearn.linear_model import LinearRegression

In [14]:
# Convert to numpy
X = g.x[:, 0].unsqueeze(1).numpy()  # Remove degree information
y = g.y.numpy()
train_mask = g.train_mask.numpy()

# Train on training split (using altitude only)
lm = LinearRegression()
reg = lm.fit(X[train_mask], y[train_mask])

# Evaluate on test split
yhat = reg.predict(X[~train_mask])
crit(torch.from_numpy(yhat), g.y[g.test_mask]).item()

186334.109375

# Results and discussion

Pulling from the results above, the Mean Squared Errors for the two models are shown below.

| GCN        | Global Linear Model |
|------------|---------------------|
| 184,472.33 | 186,334.11          |

While the error for GCN is smaller, it is likely not significant.  I am not showing error standard deviations across multiple runs here, which I may perform in the future.  For now, my curiosity was satisfies :).   Nonetheless, it is cool that GCN performs comparably out-of-the-box without tuning the number of neighbors or its model parameters. 

My curiosity was satisfied, but here are future steps I may take, and will update the blog accordingly.

## Next steps

* Explore the impact of $k$, the number of neighbors when building the graph.
* Use more thorough evaluation with multiple random splits.
* Explore different graph neural network models.
* Explore different feature engineering schemes other than appending the vertex degree.